In [ ]:
# Realizar as importações
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

input_path = 'images/'
output_path = 'output/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

# 1 - Técnicas de Meios-Tons 

## 1.1 - Definindo os Kernels

In [ ]:
kernels = {
    "floyd_steinberg": np.array([
        [   0,    0, 7/16],
        [3/16, 5/16, 1/16]
    ]),

    "stevenson_arce": np.array([
        [     0,       0,       0,      0,       0, 32/200,       0],
        [12/200,       0,  26/200,      0,  30/200,      0,  16/200],
        [     0,  12/200,       0, 26/200,       0, 12/200,       0],
        [ 5/200,       0,  12/200,      0,  12/200,      0,   5/200]
    ]),
    "burkes": np.array([
        [   0,    0,    0, 8/32, 4/32],
        [2/32, 4/32, 8/32, 4/32, 2/32]
    ]),

    "sierra": np.array([
        [   0,    0,    0, 5/32, 3/32],
        [2/32, 4/32, 5/32, 4/32, 2/32],
        [   0, 2/32, 3/32, 2/32, 0]
    ]),

    "stucki": np.array([
        [   0,    0,    0, 8/42, 4/42],
        [2/42, 4/42, 8/42, 4/42, 2/42],
        [1/42, 2/42, 4/42, 2/42, 1/42]
    ]),

    "jarvis_judice_ninke": np.array([
        [   0,    0,    0, 7/48, 5/48],
        [3/48, 5/48, 7/48, 5/48, 3/48],
        [1/48, 3/48, 5/48, 3/48, 1/48]
    ])
}

## 1.2 - Definir a função de difusão de erros

In [ ]:
def difusao(imagem, kernel, zigzag=False):
    h, w = imagem.shape
    img = imagem.copy().astype(np.float32)

    kh, kw = kernel.shape

    for y in range(h):
        if zigzag and y % 2 == 1:
            x_range = range(w-1, -1, -1)
            direcao = -1
        else:
            x_range = range(w)
            direcao = 1

        for x in x_range:
            original = img[y, x]
            novo = 0 if original < 128 else 255
            erro = original - novo
            img[y, x] = novo

            offset = kw // 2
            
            for dy in range(kh):
                for dx in range(kw):
                    nx = x + (dx - offset) * direcao
                    ny = y + dy
                    if 0 <= nx < w and 0 <= ny < h:
                        img[ny, nx] += erro * kernel[dy, dx]
    return img.astype(np.uint8)

## 1.3 - Carregar as imagens

In [ ]:
orig_g = []
nome = 'house'
imagem_gray = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
imagem_gray = imagem_gray.astype(np.float32)
orig_g.append(imagem_gray)
nome = 'city'
imagem_gray = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
imagem_gray = imagem_gray.astype(np.float32)
orig_g.append(imagem_gray)

orig_c = []
nome = 'capadocia'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_cor = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_cor = imagem_cor.astype(np.float32)
orig_c.append(imagem_cor)
nome = 'peppers'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_cor = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_cor = imagem_cor.astype(np.float32)
orig_c.append(imagem_cor)

## 1.4 - Aplicar as Técnicas de Meios-Tons

In [ ]:
imagens_g = []
titulos_g = []
imagens_c = []
titulos_c = []

for i in range(len(orig_g)):
    img_res = []
    i_titulo = []
    for nome, kernel in kernels.items():
        result = difusao(orig_g[i], kernel, zigzag=True)
        img_res.append(result)
        i_titulo.append(nome.replace("_", " ").title())
    imagens_g.append(img_res)
    titulos_g.append(i_titulo)

for i in range(len(orig_c)):
    img_res = []
    i_titulo = []
    for nome, kernel in kernels.items():
        bands = []
        for y in range(orig_c[i].shape[2]):
            result = difusao(orig_c[i][:,:,y], kernel, zigzag=True)
            bands.append(result)
        img_res.append(cv2.merge(bands))
        i_titulo.append(nome.replace("_", " ").title())
    imagens_c.append(img_res)
    titulos_c.append(i_titulo)

In [ ]:
for i in range(len(imagens_g)):
    for y in range(((len(imagens_g[i])-1) //2) + 1):
        plt.figure(figsize=(60, 25))
        plt.subplot(1, 3, 1)
        plt.imshow(orig_g[i], cmap='gray')
        plt.title(f"Imagem Original {i}", fontsize=30)
        plt.axis('off')
        for z in range(2):
            plt.subplot(1, 3, z + 2)
            plt.imshow(imagens_g[i][2*y + z], cmap='gray')
            plt.title(titulos_g[i][2*y + z], fontsize=30)
            plt.axis('off')
            cv2.imwrite(output_path + f"1_gray_{i}_" + titulos_g[i][2*y + z].replace(" ", "_") + ".png", imagens_g[i][2*y + z], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()

for i in range(len(imagens_c)):
    for y in range(((len(imagens_c[i])-1) //2) + 1):
        plt.figure(figsize=(60, 25))
        plt.subplot(1, 3, 1)
        plt.imshow(orig_c[i] / 255)
        plt.title(f"Imagem Original {i+len(imagens_g)}", fontsize=30)
        plt.axis('off')
        for z in range(2):
            plt.subplot(1, 3, z + 2)
            plt.imshow(imagens_c[i][2*y + z])
            plt.title(titulos_c[i][2*y + z], fontsize=30)
            plt.axis('off')
            imagens_c[i][2*y + z] = cv2.cvtColor(imagens_c[i][2*y + z], cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path + f"1_cor_{i+len(imagens_g)}_" + titulos_c[i][2*y + z].replace(" ", "_") + ".png", imagens_c[i][2*y + z], [cv2.IMWRITE_PNG_COMPRESSION, 0])
            imagens_c[i][2*y + z] = cv2.cvtColor(imagens_c[i][2*y + z], cv2.COLOR_BGR2RGB)
    plt.show()

# 2 - Filtragem no Domínio da Frequência  e Compressão de imagem

## 2.1 - Filtragem no Domínio da Frequência

### 2.1.1 - Abertura da imagem e aplicação da transformada

In [ ]:
nome = 'house'
imagem_gray = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
imagem_gray = imagem_gray.astype(np.float32)

freq = np.fft.fft2(imagem_gray)
freq = np.fft.fftshift(freq)

magnitude_spectrum = np.log(1 + np.abs(freq))

plt.imshow(magnitude_spectrum, cmap='gray')
plt.title('Espectro de Magnitude')
plt.axis('off')
plt.savefig(output_path + 'espectro_magnitude.png', bbox_inches='tight', pad_inches=0.1)
plt.show()

### 2.1.2 - Definição dos filtros

In [ ]:
linhas, colunas = imagem_gray.shape
clin, ccol = linhas // 2 , colunas // 2

filtros = [[0 for _ in range(3)] for _ in range(2)] + [[[0 for _ in range(3)] for _ in range(3)] for _ in range(2)]
nome_filtros = ['Passa-baixa', 'Passa-alta', 'Passa-faixa', 'Rejeita-faixa']
raio = [30, 100, 250]
medias = [30, 100, 200]
larguras = [10, 20, 50]

# Passa-baixa
for i in range(3):
    filtros[0][i] = np.zeros((linhas, colunas), np.uint8)
    cv2.circle(filtros[0][i], (ccol, clin), raio[i], 1, thickness=-1)

# Passa-alta
for i in range(3):
    filtros[1][i] = np.ones((linhas, colunas), np.uint8)
    cv2.circle(filtros[1][i], (ccol, clin), raio[i], 0, thickness=-1)

# Passa-faixa
for i in range(3):
    for j in range(3):
        r_ext = medias[i] + larguras[j]
        r_int = medias[i] - larguras[j]
        filtros[2][i][j] = np.zeros((linhas, colunas), np.uint8)
        cv2.circle(filtros[2][i][j], (ccol, clin), r_ext, 1, thickness=-1)
        filtros[2][i][j] = cv2.circle(filtros[2][i][j], (ccol, clin), max(r_int, 5), 0, thickness=-1)

# Rejeita-faixa
for i in range(3):
    for j in range(3):
        r_ext = medias[i] + larguras[j]
        r_int = medias[i] - larguras[j]
        filtros[3][i][j] = np.ones((linhas, colunas), np.uint8)
        cv2.circle(filtros[3][i][j], (ccol, clin), r_ext, 0, thickness=-1)
        filtros[3][i][j] = cv2.circle(filtros[3][i][j], (ccol, clin), max(r_int, 5), 1, thickness=-1)

### 2.1.3 - Filtragens

In [ ]:
nucleos = [[0 for _ in range(3)] for _ in range(2)] + [[[0 for _ in range(3)] for _ in range(3)] for _ in range(2)]
imagens = [[0 for _ in range(3)] for _ in range(2)] + [[[0 for _ in range(3)] for _ in range(3)] for _ in range(2)]

for i in range(2):
    for j in range(len(filtros[i])):
        nucleos[i][j] = freq * filtros[i][j]
        imagens[i][j] = np.fft.ifft2(nucleos[i][j])
        nucleos[i][j] = np.log(1 + np.abs(nucleos[i][j]))
        imagens[i][j] = np.abs(imagens[i][j])

for i in range(2,4):
    for j in range(len(filtros[i])):
        for z in range(3):
            nucleos[i][j][z] = freq * filtros[i][j][z]
            imagens[i][j][z] = np.fft.ifft2(nucleos[i][j][z])
            nucleos[i][j][z] = np.log(1 + np.abs(nucleos[i][j][z]))
            imagens[i][j][z] = np.abs(imagens[i][j][z])

### 2.1.4 - Visualização da filtragem

In [ ]:
for i in range(2):
    plt.figure(figsize=(30, 20))
    for j in range(len(nucleos[i])):
        plt.subplot(2, 3, j + 1)
        plt.imshow(nucleos[i][j], cmap='gray')
        plt.title(f'Núcleo {nome_filtros[i]} Raio: {raio[j]}')
        plt.axis('off')
        plt.subplot(2, 3, j + 1 + 3)
        plt.imshow(imagens[i][j], cmap='gray')
        plt.title(f'Filtro {nome_filtros[i]} Raio: {raio[j]}')
        plt.axis('off')
        cv2.imwrite(output_path + f"2_ncl_{nome_filtros[i]}_r{raio[j]}.png", nucleos[i][j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        cv2.imwrite(output_path + f"2_img_{nome_filtros[i]}_r{raio[j]}.png", imagens[i][j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.savefig(output_path + f"2_fig_{nome_filtros[i]}_r{'_'.join(map(str, raio))}.png", bbox_inches='tight')
    plt.show()
for i in range(2, 4):
    for j in range(len(nucleos[i])):
        plt.figure(figsize=(30, 20))
        for z in range(3):
            plt.subplot(2, 3, z + 1)
            plt.imshow(nucleos[i][j][z], cmap='gray')
            plt.title(f'Núcleo {nome_filtros[i]} Raio externo: {medias[j] + larguras[z]} Raio interno: {max(medias[j] - larguras[z], 5)}')
            plt.axis('off')
            plt.subplot(2, 3, z + 1 + 3)
            plt.imshow(imagens[i][j][z], cmap='gray')
            plt.title(f'Filtro {nome_filtros[i]} Raio externo: {medias[j] + larguras[z]} Raio interno: {max(medias[j] - larguras[z], 5)}')
            plt.axis('off')
            cv2.imwrite(output_path + f"2_ncl_{nome_filtros[i]}_re{medias[j] + larguras[z]}_ri{max(medias[j] - larguras[z], 5)}.png", nucleos[i][j][z], [cv2.IMWRITE_PNG_COMPRESSION, 0])
            cv2.imwrite(output_path + f"2_img_{nome_filtros[i]}_re{medias[j] + larguras[z]}_ri{max(medias[j] - larguras[z], 5)}.png", imagens[i][j][z], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        plt.savefig(output_path + f"2_fig_{nome_filtros[i]}_media{medias[j]}.png", bbox_inches='tight')
    plt.show()

## 2.2 - Compressão via Transformada de Fourrier

### 2.2.1 - Preparação das imagens e definição dos Thresholds

In [ ]:
thr_perc = [0.001 * i for i in range(6)] + [0.01, 0.02, 0.05]
thr_real = [thr * np.max(np.abs(freq)) for thr in thr_perc]
comprimidas = [freq.copy() for _ in range(len(thr_perc))]
img_comp = [0 for _ in range(len(thr_perc))]

### 2.2.2 - Processo de compressão

In [ ]:
for i in range(len(thr_perc)):
    comprimidas[i][np.abs(comprimidas[i]) < thr_real[i]] = 0
    img_comp[i] = np.fft.ifft2(comprimidas[i])
    comprimidas[i] = np.log(1 + np.abs(comprimidas[i]))
    img_comp[i] = np.abs(img_comp[i])

### 2.2.3 - Visualização das imagens e histogramas

In [ ]:
for i in range(len(thr_perc)):
    plt.figure(figsize=(30, 10), layout='tight')
    plt.subplot(1, 3, 1)
    plt.imshow(comprimidas[i], cmap='gray')
    plt.title(f'Espectro de Magnitude - {thr_perc[i]*100}%')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(img_comp[i], cmap='gray')
    plt.title(f'Imagem - {thr_perc[i]*100}%')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.hist(img_comp[i].ravel(), bins=256, range=[0, 256], color='blue')
    plt.title(f'Histograma - Threshold {thr_perc[i]*100}%')
    cv2.imwrite(output_path + f"3_ncl_thr{thr_perc[i]*100}.png", comprimidas[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    cv2.imwrite(output_path + f"3_img_thr{thr_perc[i]*100}.png", img_comp[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.savefig(output_path + f"3_fig_thr_{thr_perc[i]*100}.png")
    plt.show()